KLUE-BERT 이것은 KLUE팀에서 만든 한국어 BERT

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import urllib.request
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
os.chdir(r"c:\projects\PY_MLE")

데이터 로드

In [2]:
train_ner_df = pd.read_csv("ner_train_data.csv")
train_ner_df[:5]

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [3]:
test_ner_df = pd.read_csv("ner_test_data.csv")
test_ner_df[:5]

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...


In [4]:
print(len(train_ner_df))
print(len(test_ner_df))

81000
9000


In [5]:
# 문장과 레이블을 분리해서 저장. 문장도 하나의 리스트안에 토큰화 함
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [sent.split() for sent in train_ner_df['Tag'].values]
test_data_label = [sent.split() for sent in test_ner_df['Tag'].values]


In [6]:
# 예를 들어서 2번 인덱스것을 찍어본다.
print(train_data_sentence[2])
print(train_data_label[2])

['슈나이더의', '프레젠테이션은', '말', '청중을', '위한', '특별한', '쇼다', '.']
['PER-B', 'O', 'O', 'CVL-B', 'O', 'O', 'O', 'O']


In [7]:
# 개체명을 정의하고 있는 텍스트 파일을 읽어서 확인
labels = [label.strip() for label in open("ner_label.txt", 'r', encoding='utf-8')]
print(labels)

['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


In [8]:
# 개체명 태깅정보와 정수로 매핑
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}
print(tag_to_index)
print(index_to_tag)

{'O': 0, 'PER-B': 1, 'PER-I': 2, 'FLD-B': 3, 'FLD-I': 4, 'AFW-B': 5, 'AFW-I': 6, 'ORG-B': 7, 'ORG-I': 8, 'LOC-B': 9, 'LOC-I': 10, 'CVL-B': 11, 'CVL-I': 12, 'DAT-B': 13, 'DAT-I': 14, 'TIM-B': 15, 'TIM-I': 16, 'NUM-B': 17, 'NUM-I': 18, 'EVT-B': 19, 'EVT-I': 20, 'ANM-B': 21, 'ANM-I': 22, 'PLT-B': 23, 'PLT-I': 24, 'MAT-B': 25, 'MAT-I': 26, 'TRM-B': 27, 'TRM-I': 28}
{0: 'O', 1: 'PER-B', 2: 'PER-I', 3: 'FLD-B', 4: 'FLD-I', 5: 'AFW-B', 6: 'AFW-I', 7: 'ORG-B', 8: 'ORG-I', 9: 'LOC-B', 10: 'LOC-I', 11: 'CVL-B', 12: 'CVL-I', 13: 'DAT-B', 14: 'DAT-I', 15: 'TIM-B', 16: 'TIM-I', 17: 'NUM-B', 18: 'NUM-I', 19: 'EVT-B', 20: 'EVT-I', 21: 'ANM-B', 22: 'ANM-I', 23: 'PLT-B', 24: 'PLT-I', 25: 'MAT-B', 26: 'MAT-I', 27: 'TRM-B', 28: 'TRM-I'}


In [9]:
tag_size = len(tag_to_index)
print(tag_size)

29


전처리

In [10]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [11]:
# 전처리 하기 전 하나의 데이터를 가지고 샘플로 하나 만들어 보자
tokens = []
labels_ids = []


# train_data_sentence, train_data_label 둘다 리스트이다. zip으로 묶었기 때문에 요소끼리 하나의 쌍이 된다.
for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
    subword_tokens = tokenizer.tokenize(one_word)
    tokens.extend(subword_tokens)
    
    # todo  이문장이 놀랍다.
    labels_ids.extend([tag_to_index[label_token]] + [-100] * (len(subword_tokens)-1)) # [0, -100] 이런식이다.
    
print('토큰화 후 문장:', tokens)
print('레이블:', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('레이블의 정수 인코딩:', labels_ids)
print('문장의 길이:', len(tokens)) # 이걸 문장의 길이라고 할 수 있나. ## 이 있는데
print('레이블의 길이:',len(labels_ids))

토큰화 후 문장: ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레이블: ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레이블의 정수 인코딩: [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
문장의 길이: 16
레이블의 길이: 16


In [12]:
# 전처리를 위한 함수
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))


        # CLS, SEP 토큰을 추가해야 하니까. 최대길이를 max_seq_len - 2로 맞춤
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        # SEP를 추가하는 코드, tokens에는 서브워드 된 하나의 문장에 대한 단어들이 요소임
        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label] # 맨마지막(SEP)에 상응한는 -100을 추그
        
        
        # CLS를 맨앞에 추가
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids


        # 정수인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        
        # 어텐션 마스크 생성
        attention_mask = [1] * len(input_id)
        
        # 패딩시 정수인코딩 제외한 개수 
        padding_count = max_seq_len - len(input_id)

        # 정수인코딩을 제외한 나머지 패딩
        input_id = input_id + ([pad_token_id] * padding_count)
        
        # 어텐션 마스크에도 정수(글자)외에 0으로 처리
        attention_mask = attention_mask + ([0] * padding_count)
        
        # 세그먼트 인코딩
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)


        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [13]:
# 훈련데이터를 전처리 함
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 81000/81000 [00:23<00:00, 3389.87it/s]


In [14]:
# 테스트 데이터를 전처리 함
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

  0%|          | 0/9000 [00:00<?, ?it/s]

100%|██████████| 9000/9000 [00:02<00:00, 3550.17it/s]


In [24]:
# 전처리를 샘플로 한번찍어보자
print('기존 원문:', train_data_sentence[0])
print('기존 레이블:', train_data_label[0])
print('-' * 50)

# X_train은 이미 서브토큰화 되어 정수 인코딩까지 되어 있는 상태임. 이것을 디코더화 해서 보여줌
print('토큰화 후 원문:', [tokenizer.decode([word]) for word in X_train[0][0]])

# 라벨
print('토큰화 후 라벨:', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in y_train[0]])

print('-' * 50)

# 이것은 패딩까지 마친 상태임. X_train 정수인코딩, 세그먼트, 어텐션마스크까지 있다보니[][] 형태임
print('정수 인코딩 결과:', X_train[0][0])
print('정수 인코딩 라벨:', y_train[0])

기존 원문: ['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
기존 레이블: ['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
--------------------------------------------------
토큰화 후 원문: ['[CLS]', '정은', '씨', '##를', '힘들', '##게', '한', '가스', '##나', '##그', ',', '가만', '##둘', '수', '없', '##겠', '##죠', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [ ]:
# 세그먼트와 어텐션도 출력해보자
print('어텐션 마스크:', X_train[1][0])
print('세그먼트:', X_train[2][0]) # 문장이 하나니까 모두 0 으로 출력

어텐션 마스크: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


모델링

In [26]:
# 모델링 클래스를 정의하자. 여기서는 활성화 함수 대신 손실함수에서 처리하도록 한다. 실무에서는 그렇게 하지 않겠지
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt = True)
        
        # 아래는 결국 이웃풋 텐서플 포함한 레이어 또는 행렬로 볼 수 있는데 이것 자체가 하나의 객체로 보면 된다.
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras. initializers.TruncatedNormal (0.02),
                                                name='classifier')
        
    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs # inputs은 세개의 층을 포함한다.
        
        # self.bert는 모델이다. bert의 개체명인식을 위한 모델이기 때문에 아래와 같이 인자를 넣어준다.
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        # 아래가 중요하다.
        all_output = outputs[0] # outputs[1]은 many-to-one일때 사용한다. 모델의 결과가 그러니까 따지고 들 필요 없다.
        prediction = self.classifier(all_output)
        
        return prediction
        

손실함수를 정의하자

In [48]:
# 여기가 중요한데. -100은 제거하자. 즉 서브워드된 단어들은 제거한다는 의미이다.
# shape_list, reshape(-1,형태) 이 함수들은 따로 알아보자. 특히 reshape에 -1은 알아서 형태 맞게 재배열하라는 의미이다.
# 이럴겨우 차원이 한차원 줄어듬...이 부분이 정확히 이해는 안됨.

def compute_loss(labels, logits):

  # 다중클래스에서 소프트맥스 함수를 사용하지 않을 경우 from_logits=True 이렇게 해줘야 한다네...참 규칙도 더럽게는 많네
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  
  active_loss = tf.reshape(labels, (-1,)) != -100
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

  return loss_fn(labels, reduced_logits)

학습

In [49]:
model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=compute_loss)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

In [53]:
X_test

(array([[    2,   942,  2747, ...,     0,     0,     0],
        [    2,    25,  2252, ...,     0,     0,     0],
        [    2,    17, 18012, ...,     0,     0,     0],
        ...,
        [    2,  1443,  2394, ...,     0,     0,     0],
        [    2,  1327,  2025, ...,     0,     0,     0],
        [    2,  1675,  2043, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [64]:
# 다음은 콜백함수를 재정의 합니다. 기존 함수를 써도 되는데 굳이 이렇게까지...
# todo 이함수는 한번의 에포크가 끝날때마다 호출되어서 체크하게 되어 있습니다. 규칙이 그래..
# todo 중요한 것은 정수시퀀스가 아니라 실제 태깅정보로 비교한다는 것

class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
    
    # 여기가 정수시퀀스가 아니라 실제 태깅정보로 변환. f1스코어를 구하기 위해서는 정수시퀀스로 안됨
    def sequences_to_tags(self, label_ids, pred_ids):
        label_list = []
        pred_list = []
        
        for i in range(0, len(label_ids)):
            label_tag = []
            pred_tag =[]
            
            # 레이블 값이 -100은 것은 f1 score 계산 시에도 제외
            for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
                if label_index != -100:
                    label_tag.append(index_to_tag[label_index])
                    pred_tag.append(index_to_tag[pred_index])
                    
            label_list.append(label_tag)
            pred_list.append(pred_tag)
            
        return label_list, pred_list
    
    # 여기가 하나의 에포크가 끝날때마다 호출되는 함수. 콜백함수 상속을 받았기 때문에 규칙이야. 따질 필요 없음
    def on_epoch_end(self, epoch, logs={}):
        
        # 리턴값이 3차원 백터로 리턴 한갑네...1차원:한문장. 2차원:한단어 3차원:한단어에 대한 라벨값 배열 이렇게 생각하자 머리아프다
        y_predicted = self.model.predict(self.X_test)  
        y_predicted = np.argmax(y_predicted, axis = 2) # 3차원 배열에서 세번째 축에서 가장 큰 값의 인덱스 리턴
        
        label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)
        score = f1_score(label_list, pred_list, suffix=True)
        print(' - f1: {:04.2f}'.format(score * 100))
        print(classification_report(label_list, pred_list, suffix=True))

In [65]:
f1_score_report = F1score(X_test, y_test)

In [ ]:
model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks = [f1_score_report]
)

In [ ]:
# 임의의 문장에 대한 전처리 함수. 훈련시 차이점은 라벨이 존재하지 않는다는 것이다.
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            
            # 하나의 문장에 대한 서브워드 토큰화
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            
            # 서브워드 중 첫번째 단어만 0으로 채우고 나머지는 -100으로 채운다.
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # 전체길이중 cls,sep을 자리를 위해 2칸을 비운다.
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        # 토큰화 되어 있는 맨 마지막에 SEP을 추가
        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        # 토큰화 되어 있는 맨 앞에 CLS 토큰 추가
        tokens = [cls_token] + tokens
        
        # 토큰되어 있는 맨 앞쪽에 -100추가, 여기는 라벨 마스크야.
        label_mask = [pad_token_id_for_label] + label_mask

        # 토큰화 되어 있는 것을 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        
        # 어텐션 마스크
        attention_mask = [1] * len(input_id)
        
        # 패딩으로 채울 개수 산정
        padding_count = max_seq_len - len(input_id)
        
        # 정수인코딩 된 것 뒤에 패딩토큰으로 채우기
        input_id = input_id + ([pad_token_id] * padding_count)
        
        # 어텐션도 마찬가지로 나머지는 패팅토큰으로 채우기
        attention_mask = attention_mask + ([0] * padding_count)

        # 세그먼트 토큰
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        
        # 라벨마스크에 패딩길이만큼 -100으로 채운다. 라벨마스크는 정수인코딩을 0, -100으로만 인코딩 함
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

       
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [68]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 714.65it/s]


In [72]:
# 테스트 해보자
print('기존 원문  :', test_data_sentence[0])
print('-' * 50)
print('토큰화 후 원 문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레이블 마스크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])

기존 원문  : ['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼']
--------------------------------------------------
토큰화 후 원 문 : ['[CLS]', '라', '##티', '##은', '-', '원', '##윤', '##정', ',', '휘', '##닉스', '##파크', '##클', '##래', '##식', '프로', '##골', '##퍼', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

예측

In [ ]:
# 예측 함수
def ner_prediction(examples, max_seq_len, tokenizer):
    
  examples = [sent.split() for sent in examples]
  
  # 전처리
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  
  # 3차원으로 값을 리턴하니까
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  #-100 값을 삭제해서 디코딩헤서 pred_list에 추가
  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  # 토큰화된 문장의 단어마다 개체명을 묶어서 반납
  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list